```
hbase：
数据模型：
NameSpace：类似于关系型数据库中的db
RowKey：类似于关系型数据库中的主键，区别在于，rowkey是固有的
ColumnFamily：列族，可以存储多个key-value
ColumnQualifier：列修饰符，对应key-value中的key
列：由ColumnFamily和ColumnQualifier组成
时间戳：用于标识数据，获取历史数据
Region：表增大后会自动划分为多个Region

drop table if exists dws_new_mid_day;
create external table dws_new_mid_day
(
    `mid_id` string COMMENT '设备唯一标识',
    `user_id` string COMMENT '用户标识', 
    `version_code` string COMMENT '程序版本号', 
    `version_name` string COMMENT '程序版本名', 
    `lang` string COMMENT '系统语言', 
    `source` string COMMENT '渠道号', 
    `os` string COMMENT '安卓系统版本', 
    `area` string COMMENT '区域', 
    `model` string COMMENT '手机型号', 
    `brand` string COMMENT '手机品牌', 
    `sdk_version` string COMMENT 'sdkVersion', 
    `gmail` string COMMENT 'gmail', 
    `height_width` string COMMENT '屏幕宽高',
    `app_time` string COMMENT '客户端日志产生时的时间',
    `network` string COMMENT '网络模式',
    `lng` string COMMENT '经度',
    `lat` string COMMENT '纬度',
    `create_date`  string  comment '创建时间' 
)  COMMENT '每日新增设备信息'
stored as parquet
location '/warehouse/gmall/dws/dws_new_mid_day/';


insert into table dws_new_mid_day
select  
    ud.mid_id,
    ud.user_id , 
    ud.version_code , 
    ud.version_name , 
    ud.lang , 
    ud.source, 
    ud.os, 
    ud.area, 
    ud.model, 
    ud.brand, 
    ud.sdk_version, 
    ud.gmail, 
    ud.height_width,
    ud.app_time,
    ud.network,
    ud.lng,
    ud.lat,
    '2019-06-06'
from dws_uv_detail_day ud left join dws_new_mid_day nm on ud.mid_id=nm.mid_id
where ud.dt='2019-06-06' and nm.mid_id is null;


drop table if exists ads_new_mid_count;
create external table ads_new_mid_count
(
 `create_date` string COMMENT '创建时间' ,
 `new_mid_count` BIGINT COMMENT '新增设备数量' 
)  COMMENT '每日新增设备信息数量'
row format delimited fields terminated by '\t'
location '/warehouse/gmall/ads/ads_new_mid_count/';

insert into table ads_new_mid_count 
select
create_date,
count(*)
from dws_new_mid_day
where create_date='2019-06-06'
group by create_date;


drop table if exists dws_user_retention_day;
create external table dws_user_retention_day 
(
 `mid_id` string COMMENT '设备唯一标识',
 `user_id` string COMMENT '用户标识', 
 `version_code` string COMMENT '程序版本号', 
 `version_name` string COMMENT '程序版本名', 
 `lang` string COMMENT '系统语言', 
 `source` string COMMENT '渠道号', 
 `os` string COMMENT '安卓系统版本', 
 `area` string COMMENT '区域', 
 `model` string COMMENT '手机型号', 
 `brand` string COMMENT '手机品牌', 
 `sdk_version` string COMMENT 'sdkVersion', 
 `gmail` string COMMENT 'gmail', 
 `height_width` string COMMENT '屏幕宽高',
 `app_time` string COMMENT '客户端日志产生时的时间',
 `network` string COMMENT '网络模式',
 `lng` string COMMENT '经度',
 `lat` string COMMENT '纬度',
 `create_date` string comment '设备新增时间',
 `retention_day` int comment '截止当前日期留存天数'
)  COMMENT '每日用户留存情况'
PARTITIONED BY (`dt` string)
stored as parquet
location '/warehouse/gmall/dws/dws_user_retention_day/'
;

insert overwrite table dws_user_retention_day
partition(dt="2019-06-07")
select  
 nm.mid_id,
 nm.user_id , 
 nm.version_code , 
 nm.version_name , 
 nm.lang , 
 nm.source, 
 nm.os, 
 nm.area, 
 nm.model, 
 nm.brand, 
 nm.sdk_version, 
 nm.gmail, 
 nm.height_width,
 nm.app_time,
 nm.network,
 nm.lng,
 nm.lat,
nm.create_date,
1 retention_day 
from dws_uv_detail_day ud join dws_new_mid_day nm on ud.mid_id =nm.mid_id where ud.dt='2019-06-07' and nm.create_date=date_add('2019-06-07',-1);


insert overwrite table dws_user_retention_day
partition(dt="2019-06-07")
select
    nm.mid_id,
    nm.user_id,
    nm.version_code,
    nm.version_name,
    nm.lang,
    nm.source,
    nm.os,
    nm.area,
    nm.model,
    nm.brand,
    nm.sdk_version,
    nm.gmail,
    nm.height_width,
    nm.app_time,
    nm.network,
    nm.lng,
    nm.lat,
    nm.create_date,
    1 retention_day 
from dws_uv_detail_day ud join dws_new_mid_day nm  on ud.mid_id =nm.mid_id 
where ud.dt='2019-06-07' and nm.create_date=date_add('2019-06-07',-1)

union all
select  
    nm.mid_id,
    nm.user_id , 
    nm.version_code , 
    nm.version_name , 
    nm.lang , 
    nm.source, 
    nm.os, 
    nm.area, 
    nm.model, 
    nm.brand, 
    nm.sdk_version, 
    nm.gmail, 
    nm.height_width,
    nm.app_time,
    nm.network,
    nm.lng,
    nm.lat,
    nm.create_date,
    2 retention_day 
from  dws_uv_detail_day ud join dws_new_mid_day nm   on ud.mid_id =nm.mid_id 
where ud.dt='2019-06-07' and nm.create_date=date_add('2019-06-07',-2)

union all
select  
    nm.mid_id,
    nm.user_id , 
    nm.version_code , 
    nm.version_name , 
    nm.lang , 
    nm.source, 
    nm.os, 
    nm.area, 
    nm.model, 
    nm.brand, 
    nm.sdk_version, 
    nm.gmail, 
    nm.height_width,
    nm.app_time,
    nm.network,
    nm.lng,
    nm.lat,
    nm.create_date,
    3 retention_day 
from  dws_uv_detail_day ud join dws_new_mid_day nm   on ud.mid_id =nm.mid_id 
where ud.dt='2019-06-07' and nm.create_date=date_add('2019-06-07',-3);

select retention_day , count(*) from dws_user_retention_day group by retention_day;

drop table if exists ads_user_retention_day_count;
create external table ads_user_retention_day_count 
(
 `create_date` string  comment '设备新增日期',
 `retention_day` int comment '截止当前日期留存天数',
 `retention_count` bigint comment  '留存数量'
)  COMMENT '每日用户留存情况'
row format delimited fields terminated by '\t'
location '/warehouse/gmall/ads/ads_user_retention_day_count/';

insert into table ads_user_retention_day_count 
select
    create_date,
    retention_day,
    count(*) retention_count
from dws_user_retention_day
where dt='2019-06-07' 
group by create_date,retention_day;

drop table if exists ads_user_retention_day_rate;
create external table ads_user_retention_day_rate 
(
 `stat_date` string comment '统计日期',
 `create_date` string  comment '设备新增日期',
 `retention_day` int comment '截止当前日期留存天数',
 `retention_count` bigint comment  '留存数量',
 `new_mid_count` bigint comment '当日设备新增数量',
 `retention_ratio` decimal(10,2) comment '留存率'
)  COMMENT '每日用户留存情况'
row format delimited fields terminated by '\t'
location '/warehouse/gmall/ads/ads_user_retention_day_rate/';

insert into table ads_user_retention_day_rate
select 
    '2019-06-07', 
    ur.create_date,
    ur.retention_day, 
    ur.retention_count, 
    nc.new_mid_count,
    ur.retention_count/nc.new_mid_count*100
from 
(
    select
        create_date,
        retention_day,
        count(*) retention_count
    from dws_user_retention_day
    where dt='2019-06-07' 
    group by create_date,retention_day
) ur join ads_new_mid_count nc on nc.create_date=ur.create_date;


ALTER TABLE ods_order_info PARTITION (dt='2019-02-14') SET LOCATION "/warehouse/gmall/ods/ods_order_info/dt=2019-06-07";

ALTER TABLE ods_order_info PARTITION (dt='2019-02-14') RENAME TO PARTITION (dt='2019-06-07');

insert into table ads_gmv_sum_day

select
'2019-02-10' dt,
    sum(order_count) gmv_count,
    sum(order_amount) gmv_amount,
    sum(payment_amount) payment_amount 
from dws_user_action
where dt ='2019-06-06'
group by dt
;
```